In [13]:
# Her sütundaki eksik veri sayısını bul
print(data.isnull().sum())


review       0
sentiment    0
dtype: int64


In [12]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


data=pd.read_csv('IMDB Dataset.csv.zip')
#print(data.head())
#data.info()
print(data.columns)
print(data['sentiment'].value_counts())
# Her sütundaki eksik veri sayısını bul
print(data.isnull().sum())

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kurty\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kurty\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kurty\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kurty\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Index(['review', 'sentiment'], dtype='object')
sentiment
positive    25000
negative    25000
Name: count, dtype: int64


In [14]:
# NLTK kütüphanesinin stop-word'lerini ve lemmatizer'ını hazırla
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    # 1. HTML etiketlerini kaldır
    text = re.sub(r'<.*?>', '', text)
    # 2. Noktalama işaretlerini ve rakamları kaldır
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # 3. Metni küçük harfe çevir
    text = text.lower()
    # 4. Kelimeleri ayır (Tokenization)
    words = word_tokenize(text)
    # 5. Stop-word'leri kaldır ve kelimeleri köklerine indir (Lemmatization)
    filtered_words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    # 6. Temizlenmiş kelimeleri tekrar birleştir
    return " ".join(filtered_words)

In [15]:
import nltk

try:
    nltk.download('punkt_tab')
    print("punkt_tab verisi başarıyla indirildi.")
except Exception as e:
    print("İndirme başarısız oldu. Hata:", e)
    

punkt_tab verisi başarıyla indirildi.


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\kurty\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [16]:
# Veri setinden rastgele bir yorum seçelim (örneğin ilk yorum)
orijinal_yorum = data['review'][0]

# Yazdığın clean_text fonksiyonu ile yorumu temizle
temizlenmis_yorum = clean_text(orijinal_yorum)

print("--- Orijinal Yorum ---")
print(orijinal_yorum)

print("\n--- Temizlenmiş Yorum ---")
print(temizlenmis_yorum)

--- Orijinal Yorum ---
One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the

In [17]:
# 'clean_text' fonksiyonunu 'review' sütununa uygula
data['cleaned_review'] = data['review'].apply(clean_text)

# Temizlenmiş verinin ilk 5 satırını göster
print(data[['review', 'cleaned_review', 'sentiment']].head())

                                              review  \
0  One of the other reviewers has mentioned that ...   
1  A wonderful little production. <br /><br />The...   
2  I thought this was a wonderful way to spend ti...   
3  Basically there's a family where a little boy ...   
4  Petter Mattei's "Love in the Time of Money" is...   

                                      cleaned_review sentiment  
0  one reviewer mentioned watching oz episode you...  positive  
1  wonderful little production filming technique ...  positive  
2  thought wonderful way spend time hot summer we...  positive  
3  basically there family little boy jake think t...  negative  
4  petter matteis love time money visually stunni...  positive  


In [18]:
vectorizer = TfidfVectorizer(min_df=5, max_df=0.8, stop_words='english')

In [19]:
X = vectorizer.fit_transform(data['cleaned_review'])

In [20]:
y = data['sentiment'].map({'positive': 1, 'negative': 0})
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Vektörleştirilmiş veri boyutu:", X.shape)
print(f"Eğitim seti boyutu: {X_train.shape}")
print(f"Test seti boyutu: {X_test.shape}")



Vektörleştirilmiş veri boyutu: (50000, 35925)
Eğitim seti boyutu: (40000, 35925)
Test seti boyutu: (10000, 35925)


In [22]:

# Lojistik Regresyon modelini oluştur
model = LogisticRegression(solver='liblinear')

# Modeli eğitim verileriyle eğit
model.fit(X_train, y_train)

# Test verileri üzerinde tahmin yap
y_pred = model.predict(X_test)

# Modelin doğruluk oranını ölç ve yazdır
dogruluk = accuracy_score(y_test, y_pred)
print(f"Modelin doğruluk oranı: {dogruluk:.2f}\n")

# Detaylı performans raporunu göster
print(classification_report(y_test, y_pred))

Modelin doğruluk oranı: 0.89

              precision    recall  f1-score   support

           0       0.90      0.87      0.89      4961
           1       0.88      0.91      0.89      5039

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000

